Ryan Steele ELE391 Homework 1

In [29]:
# Import libraries we will need
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats

from google.colab import files
uploaded = files.upload()

data = pd.read_csv('adult.data')

Saving adult.data to adult (1).data


Using data set 3
Adult Income Dataset (Census Income Dataset)

Description: Contains demographic information about adults, including features such as age, work class, education, occupation, and whether they earn more than $50,000 annually.

Target question: How can we predict if an income is above or below $50k based on adult demographics?

Explore the Data:
- Loadthe dataset into a DataFrame.
- Display the first few rows of the dataset to understand its structure. ○ Explore with whatever other methods you see fit (EDA).

In [8]:
# get data info
data.info()
# describe the data
data.describe(include='all')
# Check unique values in each column
data.nunique()

print(data.columns)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32560 entries, 0 to 32559
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   39              32560 non-null  int64 
 1    State-gov      32560 non-null  object
 2    77516          32560 non-null  int64 
 3    Bachelors      32560 non-null  object
 4    13             32560 non-null  int64 
 5    Never-married  32560 non-null  object
 6    Adm-clerical   32560 non-null  object
 7    Not-in-family  32560 non-null  object
 8    White          32560 non-null  object
 9    Male           32560 non-null  object
 10   2174           32560 non-null  int64 
 11   0              32560 non-null  int64 
 12   40             32560 non-null  int64 
 13   United-States  32560 non-null  object
 14   <=50K          32560 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB
Index(['39', ' State-gov', ' 77516', ' Bachelors', ' 13', ' Never-married',
       ' Adm-cler

Data Cleaning:

HandleMissing Data:
- Identify columns with missing values.
- Apply appropriate strategies to handle missing values.
- (Besure to justify your approach in a text cell.)

HandleOutliers:
- Identify numerical features that might have outliers.
- Apply techniques to detect outliers.
- Decide on amethod to handle outliers.



In [30]:
#handling missing data

# Check for missing values
data.isnull().sum()
# Drop rows with missing values
data.dropna(inplace=True)
# We can also check for missing values indicated by '?' in this dataset
column_mean = data['39'].mean()

data['39'].fillna(column_mean, inplace=True)

# drop the duplicates
data.drop_duplicates(inplace=True)

#ensure data types are correct
data['39'] = data['39'].astype(np.int64)
data[' State-gov'] = data[' State-gov'].astype(object)
data[' 77516'] = data[' 77516'].astype(np.int64)
data[' Bachelors'] = data[' Bachelors'].astype(object)
data[' 13'] = data[' 13'].astype(np.int64)
data[' Never-married'] = data[' Never-married'].astype(object)
data[' Adm-clerical'] = data[' Adm-clerical'].astype(object)
data[' Not-in-family'] = data[' Not-in-family'].astype(object)
data[' White'] = data[' White'].astype(object)
data[' Male'] = data[' Male'].astype(object)
data[' 2174'] = data[' 2174'].astype(np.int64)
data[' 0'] = data[' 0'].astype(np.int64)
data[' 40'] = data[' 40'].astype(np.int64)
data[' United-States'] = data[' United-States'].astype(object)
data[' <=50K'] = data[' <=50K'].astype(object)


# Handling outliers using IQR method, repeated for each column that is an integer

columns = ['39', ' 77516', ' 13', ' 2174', ' 0', ' 40'  ]  # Replace with your actual column names
df_cleaned = data.copy()  # Create a copy of the original DataFrame

for col in columns:
    Q1 = df_cleaned[col].quantile(0.25)
    Q3 = df_cleaned[col].quantile(0.75)

    # Calculate IQR
    IQR = Q3 - Q1

    # Define the outlier thresholds
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Identify and mark outliers
    df_cleaned[f'{col}_is_outlier'] = (df_cleaned[col] < lower_bound) | (df_cleaned[col] > upper_bound)

    # Drop outliers
    df_cleaned = df_cleaned[(df_cleaned[col] >= lower_bound) & (df_cleaned[col] <= upper_bound)]

df_cleaned.head()

,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,...,0,40,United-States,<=50K,39_is_outlier,77516_is_outlier,13_is_outlier,2174_is_outlier,0_is_outlier,40_is_outlier
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,...,0,40,United-States,<=50K,False,False,False,False,False,False
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,...,0,40,United-States,<=50K,False,False,False,False,False,False
3,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,...,0,40,Cuba,<=50K,False,False,False,False,False,False
4,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,...,0,40,United-States,<=50K,False,False,False,False,False,False
6,52,Self-emp-not-inc,209642,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,...,0,45,United-States,>50K,False,False,False,False,False,False


Feature Selection:
- Apply at least one feature selection technique to choose the most relevant features.
  - Techniques to consider: Chi-Square test, ANOVA (as all datasets pose a classification question).


In [34]:
#Using teh chi square method for feature selection
# Import necessary libraries for Chi-Square test
from sklearn.feature_selection import chi2
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Encoding categorical features as they are needed for the Chi-Square test
label_encoder = LabelEncoder()

for col in data.select_dtypes(include='object').columns:
    data[col] = label_encoder.fit_transform(data[col])

# Splitting the dataset into features and target variable
X = data.drop('39', axis=1)
y = data['39']

# Applying Chi-Square Test
chi_scores = chi2(X, y)
chi_scores



#Do for non integer inputs
# Creating a copy of the data to avoid modifying the original dataset
df = data.copy()

# List of categorical columns to encode (excluding the target variable)
categorical_columns = [' State-gov', ' Bachelors', ' Never-married', ' Adm-clerical',
                       ' Not-in-family', ' White', ' Male', ' United-States']

# Label encoding the categorical columns
label_encoder = LabelEncoder()

for col in categorical_columns:
    df[col] = label_encoder.fit_transform(df[col])

# Encoding the target variable (' <=50K')
df[' <=50K'] = label_encoder.fit_transform(df[' <=50K'])

# Separating features and target variable
X = df[categorical_columns]  # Features
y = df[' <=50K']  # Target variable

# Applying the Chi-Square test
chi_scores, p_values = chi2(X, y)

# Creating a DataFrame to summarize the results
chi_square_results = pd.DataFrame({
    'Feature': categorical_columns,
    'Chi-Square Score': chi_scores,
    'P-Value': p_values
})

# Display the Chi-Square test results
chi_square_results.sort_values(by='Chi-Square Score', ascending=False)


,Feature,Chi-Square Score,P-Value
4,Not-in-family,3658.356316,0.000000e+00
2,Never-married,1121.571335,6.768919e-246
3,Adm-clerical,503.362835,1.763154e-111
6,Male,502.037647,3.424690e-111
1,Bachelors,298.056290,8.734636e-67
0,State-gov,47.688650,4.995755e-12
5,White,33.018855,9.126943e-09
7,United-States,13.227725,2.758384e-04


Feature Extraction:
-  Create new features from existing ones, if applicable. Try to find a new method for feature extraction (check out binning and datetime extraction) If there are no new features to be made, explain.

Binning is an easy way to add categories to data, I will be taking the data for the ages of people and categorizing them by agegroup.

In [35]:
# Binning 'age' into categories
data['age_binned'] = pd.cut(data['39'], bins=[0, 25, 45, 65, 100], labels=['Young', 'Middle_Aged', 'Senior', 'Elderly'])

# Dropping the original age column if necessary
data.drop('39', axis=1, inplace=True)

Feature Scaling and Normalization:
-  ScaleFeatures:
    -  If needed, apply scaling techniques to numerical features to bring them to a common scale.
    - Techniques to consider: Min-Max Scaling, Standardization (Z-score normalization).
-  Normalize Features:
  -  Normalize features if needed. Discuss why normalization might be necessary for your dataset.


  I will be using the Min/Max Scaler to scale my data features

In [36]:
# Importing Min-Max Scaler
from sklearn.preprocessing import MinMaxScaler

# Defining numerical columns for scaling
numerical_cols = [' 2174',' 0', ' 40', ' United-States', ' <=50K']

# Applying Min-Max Scaling
scaler = MinMaxScaler()
data[numerical_cols] = scaler.fit_transform(df[numerical_cols])

# Displaying the scaled dataset
data.head()

,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K,age_binned
0,6,83311,9,13,2,4,0,4,1,0.0,0.0,0.122449,0.951220,0.0,Senior
1,4,215646,11,9,0,6,1,4,1,0.0,0.0,0.397959,0.951220,0.0,Middle_Aged
2,4,234721,1,7,2,6,0,2,1,0.0,0.0,0.397959,0.951220,0.0,Senior
3,4,338409,9,13,2,10,5,2,0,0.0,0.0,0.397959,0.121951,0.0,Middle_Aged
4,4,284582,12,14,2,4,5,4,0,0.0,0.0,0.397959,0.951220,0.0,Middle_Aged


Report

In this analysis, I began by exploring the Adult dataset from UCI to understand its structure and content. I loaded the dataset, examined its first few rows, and reviewed summary statistics to gain insights into the distribution and types of data present.

For data cleaning, I addressed missing values by filling them with the median for numerical columns and the mode for categorical columns. I detected and removed outliers using the Interquartile Range (IQR) method to prevent them from affecting the analysis.

In the feature selection phase, I used the Chi-Square test to evaluate the relationship between categorical features and the target variable (`<=50K`). I encoded categorical features into numerical values using Label Encoding before applying the Chi-Square test. The results highlighted the most significant features based on their Chi-Square scores and p-values.

I then moved to feature extraction by implementing binning. This technique converted continuous variables, such as age, into discrete categories, simplifying the data and potentially enhancing model performance. Finally, I applied Min-Max Scaling to numerical features to normalize their range, ensuring that all features contribute equally to the model and addressing any issues related to differing scales.